In [2]:
import numpy as np
import math

In [16]:
# задан портфель из двух инструментов
portfolio = [
    {
        'quantity': 1,
        'market_depth': 10_000,
        'execution_lag': 3,
        'DS': False,
        'spot': 100
    },
    {
        'quantity': -2,
        'market_depth': 10_000,
        'execution_lag': 1,
        'DS': False,
        'spot': 100
    }

]

In [13]:
# это просто риск-сценарии, количество строк независимо от количества активов,
# но предполагается что количество столбцов равно периоду ликвидации
# и вообще говоря их мы просто генерируем
r_k = np.array([[0.3, 0.5, 0.6],
                [-0.2, -0.3, -0.4],
                [0.1, -0.4, -0.2]]) + 1

def_r = np.full((len(r_k[0]), 1), 1) # default risk factor

risk_scenarios = np.hstack((def_r, r_k))
risk_scenarios


array([[1. , 1.3, 1.5, 1.6],
       [1. , 0.8, 0.7, 0.6],
       [1. , 1.1, 0.6, 0.8]])

In [17]:
def get_liquidation_period(portfolio):
    check = []
    for i in portfolio:
        check.append(i['execution_lag'] + math.ceil(i['quantity'] / i['market_depth']))

    return max(check) - 1

In [18]:
H = get_liquidation_period(portfolio)
H

3

In [19]:
# dummy formula
def check_num_of_strategies(instrument, liquidation_period):
    return math.comb(liquidation_period - instrument['execution_lag'] + abs(instrument['quantity']),
                     abs(instrument['quantity']))

In [22]:
check_num_of_strategies(portfolio[1], H)

6

In [26]:
# для первого инструмента только одна стратегия: [0 0 1]
# наверное перебрать все стратегии, но это очень неэффективно
possible_strats_for_1 = [[0, 0, 1]]
possible_strats_for_2 = [[2, 0, 0],
                         [1, 1, 0],
                         [1, 0, 1],
                         [0, 1, 1],
                         [0, 2, 0],
                         [0, 0, 2]]

In [23]:
def result_function(instrument, risk_scenarios):
    return risk_scenarios[::, instrument['execution_lag']:] * instrument['quantity'] * instrument['spot']

In [24]:
result_function(portfolio[0], risk_scenarios)

array([[160.],
       [ 60.],
       [ 80.]])

In [25]:
result_function(portfolio[1], risk_scenarios)

array([[-260., -300., -320.],
       [-160., -140., -120.],
       [-220., -120., -160.]])

![img.png](img.png)

### strategy 2 0 0
-130 * 2 --> -260 --> -260 + 160
-80 * 2  --> -160 --> -160 + 60
-110 * 2 --> -220 --> -220 + 80

### strategy 1 1 0
-130 --> -130 + -150 --> -280 + 160
 -80 --> -80 + -70   --> -150 + 60
-110 --> -110 + -60  --> -170 + 80

### strategy 1 0 1
-130 --> -130 --> -130 + -160 + 160
 -80 --> -80  --> -80 + -60 + 60
-110 --> -110 --> -110 + -80 + 80

### strategy 0 1 1
0 --> -150 --> -150 + -160 + 160
0 --> -70  --> -70 + -60 + 60
0 --> -60  --> -60 + -80 + 80

### strategy 0 2 0
0 --> -150 * 2 --> -300 + 160
0 --> -70 * 2  --> -140 + 60
0 --> -60 * 2  --> -120 + 80

### strategy 0 0 2
0 --> 0 --> -160 * 2 + 160
0 --> 0 --> -60 * 2 + 60
0 --> 0 --> -80 * 2 + 80